In [ ]:
from nbtemplate import display_header
display_header('index.ipynb')

# Ray-tracing the AXIS XGS

In these pages I present the ray-trace studies I've done so far for the AXIS x-ray grating spectrometer (XGS). My goal is to explore different design options, but since some parameters of the mission are not fully defined yet, the exact numbers may change, but, based on my experience with . The top level results are:

- A spectral resolving power of $R = 3000$ is easy. We will reach that in any design studied.

- For higher spectral resolving powers, the size of the flat gratings becomes the limiting factor, we have to use either smaller (and thus more) gratings, or bend or chirp them. With that, $R = 8000$ is possible. We choose $30*60$ mm$^2$ gratings as fiducial for most of these studies.

- Reaching an effective area of 1000 cm$^2$ at O VII requires the gratings to cover about 2/3 of the aperture. For $60*60$ mm$^2$ gratings, that requires about 360 gratings - fewer than Arcus.

- Given that the PSF is not well-suited to sup-aperturing, the ideal aperturing scheme is **not** to opposite sectors, but just a gap in the negative dispersion direction (see 3D image below). 

- The blaze peak is relatively narrow. Detectors for the dispersed spectrum need to cover about 150-200 mm of length (half the size of a single Arcus camera). We simulate 4 CCDs of $25 * 50$ mm$^2$ for more of these simulations.

- Pure Si gratings are essentially transparent above 2 keV. Even with the gratings inserted, the zeroth order still received 70% of the photons with would without gratings. The grating spectrum ranges from about 1.5-5 nm.

- Coated gratings (we simulate Pt), extend the useful range of the gratings to about 2 keV.

Plots and more detailes can be found on the other pages of this website (see menu bar on top).


Please let me know if you have any questions or need high-resolution versions of any of the plots or
the data to recreate the plot by emailing hgunther@mit.edu. (The plots here are optimized for viewing on the web.)

Here is the pointer to the software used:

- My general ray-trace code [MARXS](http://marxs.readthedocs.io/en/latest/index.html)
- Ray-trace code [specific to Lynx](https://github.com/hamogu/marxs-lynx)

## AXIS XGS: How does is look?
In this section, I show how the XGS setup looks. I start with an interactive 3D view, which can be zoomed and rotated with the mouse in all supported browsers; pressing "r" returns the view to the initial position. See [the X3DOM documentation](https://www.x3dom.org/documentation/interaction/) for a full list of supported mouse and keyboard commands.

Rays start in the aperture, which is shaped like a series of concentric rings. In the AXIS mirrors, photons will bounce twice in a Wolter-Schwarzschild geometry. However, in this simulation the mirror is simplified such that the reflection actually happens in a single plane (the the double reflection is taken into account when calculating the survival probability of a photon).  

In [ ]:
import numpy as np
import astropy.units as u
from astropy.table import Table
from astropy.coordinates import SkyCoord
from marxs.source import PointSource, FixedPointing
from marxs import simulator
from marxs.visualization.x3d import plot_object, plot_rays
from marxslynx.ralfgrating import facet_table
from marxslynx import AXIS as axis

import x3d

In [ ]:
n_photons = 1e4
wave = np.arange(6., 60., 0.5) * u.Angstrom
energies = wave.to(u.keV, equivalencies=u.spectral())
instrument = axis.AXISForPlot(conf=axis.conf)


In [ ]:
facettab = facet_table(instrument.elements[2])

In [ ]:
# Restricting facets to a certain range in angles
ind = np.abs(np.abs(facettab['facet_ang'])) < np.pi * 2 / 3
instrument.elements[2].elements = [instrument.elements[2].elements[i] for i in ind.nonzero()[0]]

In [ ]:
src = PointSource(coords=SkyCoord(30. * u.deg, 30. * u.deg),
                           energy=0.4 * u.keV,
                           flux=1. / u.s / u.cm**2)
pointing = FixedPointing(coords=SkyCoord(30 * u.deg, 30. * u.deg))

In [ ]:
keeppos = simulator.KeepCol('pos')
keepprob = simulator.KeepCol('probability')
instrument.postprocess_steps = [keeppos, keepprob]
photons = src.generate_photons(n_photons * u.s)
photons = pointing(photons)
photons = instrument(photons)

In [ ]:
#fig = mlab.figure(bgcolor=(.8, .8, .8))
#mlab.clf()

out = plot_object(instrument)

# Need to implement drawing of lines in x3d backend!!

#pos = keeppos.format_positions()
#ind = (photons['CCD_ID'] >= 0) | np.isfinite(photons['microcal_x'])
#photons['order'][np.isnan(photons['order'])] = 0
#out = plot_rays(pos[ind, :, :], scalar=photons['order'][ind], viewer=fig)

In [ ]:
out.dimension_px = (1000, 800)
out.children.append(x3d.x3d.Viewpoint(
    centerOfRotation=(5000, 0, 0),
    description='Overview',
    fieldOfView=1, # panoramic view. Default is 45 deg = 0.78 or so
    orientation=(-.2, .97, -.2, 2.4),
    position=(11000, -3000, -5000),
))
out

Basic design for a AXIS CAT grating spectrometer, showing a ray-trace for an on-axis point source with a monochromatic emission at 0.4 keV = 31 Å.
This is the scenario with the nominal gratings we studied (30 mm * 60 mm). Gratings are arbitrarily colored in sectors. Not the entire aperture is filled with gratings. We study different ways to sub-aperture and this is just one possible configuration.